<a href="https://colab.research.google.com/github/amine406/SeoProject/blob/main/Crawling_s%C3%A9quentiel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import pandas as pd

# Initialiser les variables
visited_urls = set()  # Pour stocker les URLs visitées
internal_urls = set()  # Pour stocker les URLs internes
url_info_list = []  # Stocker les infos SEO pour chaque URL

# Fonction pour vérifier si l'URL est valide et interne
def is_valid_url(url, base_url):
    parsed_url = urlparse(url)
    return bool(parsed_url.scheme) and bool(parsed_url.netloc) and base_url in url

# Fonction pour crawler et analyser une pagea
def crawl_page(url, base_url):
    try:
        response = requests.get(url)
        status_code = response.status_code
        print(f"Crawling {url} (status code: {status_code})")

        # Vérifier que la réponse est bien du HTML
        if "text/html" not in response.headers["Content-Type"]:
            print(f"Skipping non-HTML content at {url}")
            return [] # Return an empty list if not HTML content

        # Forcer l'encodage si nécessaire
        response.encoding = 'utf-8'  # Forcer l'encodage UTF-8

        # Utiliser un parser robuste
        soup = BeautifulSoup(response.content, "html.parser")  # Ou "lxml"

        # Récupérer les informations SEO
        title = soup.title.string if soup.title else "No title"
        description = soup.find("meta", attrs={"name": "description"})
        description = description["content"] if description else "No description"
        h1 = soup.find("h1")
        h1_text = h1.get_text() if h1 else "No H1"

        # Compter le nombre de liens sur la page
        links = soup.find_all("a", href=True)
        num_links = len(links)

        # Stocker les informations dans une liste
        url_info_list.append({
            "URL": url,
            "HTTP Status": status_code,
            "Title": title,
            "Meta Description": description,
            "H1": h1_text,
            "Number of Links": num_links
        })

        # Retourner la liste des liens internes
        return [urljoin(base_url, link.get('href')) for link in links if is_valid_url(urljoin(base_url, link.get('href')), base_url)]

    except requests.RequestException as e:
        print(f"Failed to crawl {url}: {e}")
        return [] # Return an empty list if there is an error

# Fonction principale pour démarrer le crawling
def crawl_website(start_url):
    base_url = "{0.scheme}://{0.netloc}".format(urlparse(start_url))
    to_crawl = [start_url]

    while to_crawl:
        url = to_crawl.pop()

        if url not in visited_urls:
            visited_urls.add(url)
            new_links = crawl_page(url, base_url)

            # Ajouter les nouveaux liens à la pile des URLs à crawler
            to_crawl.extend(new_links)

    # Convertir les données collectées en DataFrame
    df = pd.DataFrame(url_info_list)

    # Afficher le tableau
    print("\nTableau récapitulatif des pages crawlées:")
    print(df)

    # Exporter vers un fichier CSV si besoin
    df.to_csv("seo_crawl_results.csv", index=False)

# Démarrer le crawler sur le site "https://aundetailpres.fr"
if __name__ == "__main__":
    crawl_website("https://aundetailpres.fr")